# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker
!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

/home/selim/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join('..', '..', '..', "data", "secondary_tags", "augmented_specific_needs_groups.csv")

df = pd.read_csv(DATA_PATH, index_col=0, lineterminator='\n')

train_df, val_df = train_test_split(df, test_size=0.2)

In [7]:
train_df.head(2)

,entry_id,excerpt,specific_needs_groups
6048,323616,(Sahel) Para este mes se han registrado 54 nue...,"['Female Head of Household', 'Chronically Ill'..."
2902,287266,Effets des inondations Sur les moyens de subsi...,['Female Head of Household']


## Sagemaker Prep

### Session

Configure SageMaker

In [8]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

/home/selim/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Bucket upload

You need to upload data to an S3 bucket. 




In [9]:
sample = False  # To make the computations faster, sample = True.

if sample:
    train_df = train_df.sample(n=1000)
    val_df = val_df.sample(n=1000)
    
job_name = f"pytorch-{formatted_time()}-subpillars-model-test-mlflow"  # cannot be changed
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


train_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
val_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [10]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [11]:
from sagemaker.pytorch import PyTorch


hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'max_len': 512,
    'epochs': 5,
    'model_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 384,
    'learning_rate': 7e-5,
    'experiment_name': "pl-specific-needs-groups",
    'training_column':'specific_needs_groups',
    'multiclass_bool':True,
    'train_with_whole_dataset':True
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type='ml.p2.xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [12]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [13]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-08-25 10:59:59 Starting - Starting the training job...
2021-08-25 11:00:26 Starting - Launching requested ML instancesProfilerReport-1629889196: InProgress
...............
2021-08-25 11:03:07 Starting - Preparing the instances for training.........
2021-08-25 11:05:08 Downloading - Downloading input data
2021-08-25 11:05:08 Training - Downloading the training image........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-25 11:09:30,130 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-25 11:09:30,155 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-25 11:09:31,593 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-25 11:09:32,578 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.


2021-08-25 11:10:09 Training - Training image download completed. Training in progress.Collecting aiohttp>=3.3.1
  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392143 sha256=7700d81441ffd01d7c4d844f5ccb6e331c3f716cb1d2a296e71cb1b035efa4f0
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591917 sha256=f6dbe30d7fbe60aba90bcbcca052936d1cfe2a9cba1b786b9d55a4ae890df597
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.4.1-py3-none-any.whl size=49822 sha256=ab7caeb605d65bcc689854abcd3a2dde72f176f0dad8a183254c6b40e6cc9e12
  Stored in directory: /root/.cache/pip/wheels/0e/ee/c0/a79457b0bdf63d19a62fb58df7d8f760e5e4811ef2f7058436
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=fc6

  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.110
    Uninstalling botocore-1.20.110:
      Successfully uninstalled botocore-1.20.110
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.51.0
    Uninstalling tqdm-4.51.0:
      Successfully uninstalled tqdm-4.51.0
  Attempting uninstall: h5py
    Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
  Attempting uninstall: boto3
    Found e

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-08-25 11:12:12.082 algo-1:92 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-25 11:12:12.132 algo-1:92 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-08-25 11:12:12.133 algo-1:92 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-08-25 11:12:12.133 algo-1:92 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-08-25 11:12:12.134 algo-1:92 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-08-25 11:12:12.134 algo-1:92 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-08-25 11:12:12.343 algo-1:92 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:11720448
[2021-08-25 11:12:12.344 algo-1:92 INFO hook.py:594] nam

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/737 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/737 [00:00<?, ?it/s] #015Epoch 0:   4%|▍         | 30/737 [00:09<03:36,  3.27it/s]#015Epoch 0:   4%|▍         | 30/737 [00:09<03:36,  3.27it/s, loss=2, v_num=0, val_f1_epoch=0.0797, val_loss_epoch=0.713, train_f1=0.524]#015Epoch 0:   8%|▊         | 60/737 [00:17<03:22,  3.34it/s, loss=2, v_num=0, val_f1_epoch=0.0797, val_loss_epoch=0.713, train_f1=0.524]#015Epoch 0:   8%|▊         | 60/737 [00:17<03:22,  3.34it/s, loss=0.996, v_num=0, val_f1_epoch=0.0797, val_loss_epoch=0.713, train_f1=0.479]#015Epoch 0:  12%|█▏        | 90/737 [00:26<03:12,  3.37it/s, loss=0.996, v_num=0, val_f1_epoch=0.0797, val_loss_epoch=0.713, train_f1=0.479]#015Epoch 0:  12%|█▏        | 90/737 [00:26<03:12,  3.37it/s, loss=0.942, v_num=0, val_f1_epoch=0.0797, val_loss_epoch=0.713, train_f1=0.480]#015Epoch 0:  16%|█▋        | 120/

#015Validating: 100%|██████████| 67/67 [00:13<00:00,  4.92it/s]#033[A#015Epoch 1: 100%|██████████| 737/737 [03:32<00:00,  3.46it/s, loss=0.481, v_num=0, val_f1_epoch=0.858, val_loss_epoch=0.114, train_f1=0.789, val_f1_step=0.815, val_loss_step=0.136]
#015                                                           #033[A#015Epoch 1:   0%|          | 0/737 [00:00<?, ?it/s, loss=0.481, v_num=0, val_f1_epoch=0.858, val_loss_epoch=0.114, train_f1=0.789, val_f1_step=0.815, val_loss_step=0.136]          #015Epoch 2:   0%|          | 0/737 [00:00<?, ?it/s, loss=0.481, v_num=0, val_f1_epoch=0.858, val_loss_epoch=0.114, train_f1=0.789, val_f1_step=0.815, val_loss_step=0.136]#015Epoch 2:   4%|▍         | 30/737 [00:09<03:35,  3.27it/s, loss=0.481, v_num=0, val_f1_epoch=0.858, val_loss_epoch=0.114, train_f1=0.789, val_f1_step=0.815, val_loss_step=0.136]#015Epoch 2:   4%|▍         | 30/737 [00:09<03:36,  3.27it/s, loss=0.365, v_num=0, val_f1_epoch=0.858, val_loss_epoch=0.114, train_f1=0.847, val_f1_

#015Validating: 100%|██████████| 67/67 [00:13<00:00,  4.89it/s]#033[A#015Epoch 2: 100%|██████████| 737/737 [03:33<00:00,  3.46it/s, loss=0.362, v_num=0, val_f1_epoch=0.880, val_loss_epoch=0.0981, train_f1=0.899, val_f1_step=0.820, val_loss_step=0.127]
#015                                                           #033[A#015Epoch 2:   0%|          | 0/737 [00:00<?, ?it/s, loss=0.362, v_num=0, val_f1_epoch=0.880, val_loss_epoch=0.0981, train_f1=0.899, val_f1_step=0.820, val_loss_step=0.127]          #015Epoch 3:   0%|          | 0/737 [00:00<?, ?it/s, loss=0.362, v_num=0, val_f1_epoch=0.880, val_loss_epoch=0.0981, train_f1=0.899, val_f1_step=0.820, val_loss_step=0.127]#015Epoch 3:   4%|▍         | 30/737 [00:09<03:34,  3.30it/s, loss=0.362, v_num=0, val_f1_epoch=0.880, val_loss_epoch=0.0981, train_f1=0.899, val_f1_step=0.820, val_loss_step=0.127]#015Epoch 3:   4%|▍         | 30/737 [00:09<03:34,  3.30it/s, loss=0.362, v_num=0, val_f1_epoch=0.880, val_loss_epoch=0.0981, train_f1=0.837, va

#015Validating: 100%|██████████| 67/67 [00:13<00:00,  4.94it/s]#033[A#015Epoch 3: 100%|██████████| 737/737 [03:32<00:00,  3.46it/s, loss=0.316, v_num=0, val_f1_epoch=0.896, val_loss_epoch=0.0884, train_f1=0.784, val_f1_step=0.845, val_loss_step=0.116]
#015                                                           #033[A#015Epoch 3:   0%|          | 0/737 [00:00<?, ?it/s, loss=0.316, v_num=0, val_f1_epoch=0.896, val_loss_epoch=0.0884, train_f1=0.784, val_f1_step=0.845, val_loss_step=0.116]          #015Epoch 4:   0%|          | 0/737 [00:00<?, ?it/s, loss=0.316, v_num=0, val_f1_epoch=0.896, val_loss_epoch=0.0884, train_f1=0.784, val_f1_step=0.845, val_loss_step=0.116]#015Epoch 4:   4%|▍         | 30/737 [00:09<03:35,  3.29it/s, loss=0.316, v_num=0, val_f1_epoch=0.896, val_loss_epoch=0.0884, train_f1=0.784, val_f1_step=0.845, val_loss_step=0.116]#015Epoch 4:   4%|▍         | 30/737 [00:09<03:35,  3.29it/s, loss=0.282, v_num=0, val_f1_epoch=0.896, val_loss_epoch=0.0884, train_f1=0.902, va

#015Validating: 100%|██████████| 67/67 [00:13<00:00,  4.93it/s]#033[A#015Epoch 4: 100%|██████████| 737/737 [03:32<00:00,  3.46it/s, loss=0.277, v_num=0, val_f1_epoch=0.900, val_loss_epoch=0.0843, train_f1=0.845, val_f1_step=0.849, val_loss_step=0.113]
#015                                                           #033[A#015Epoch 4: 100%|██████████| 737/737 [03:34<00:00,  3.44it/s, loss=0.277, v_num=0, val_f1_epoch=0.900, val_loss_epoch=0.0843, train_f1=0.845, val_f1_step=0.849, val_loss_step=0.113]
(4049, 15)
(4049, 15)
2021-08-25 11:11:17.652753: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/08/25 11:11:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
INFO:filelock:Lock 140209473432824 acquired on /root


2021-08-25 11:30:55 Uploading - Uploading generated training model
2021-08-25 11:31:15 Completed - Training job completed
ProfilerReport-1629889196: IssuesFound
Training seconds: 1585
Billable seconds: 1585
